<a href="https://colab.research.google.com/github/jackshengxinz/Machine-Learning-Model-Fairness-Under-Demographic-Shift/blob/main/Adult_dataset_Pytorch_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load Data
from google.colab import files
uploaded = files.upload()

Saving adult_norm copy.csv to adult_norm copy.csv


In [ ]:

import torch.nn as nn
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from PIL import Image
import os
import pandas as pd
from numpy import random
import os
import torch
from sklearn import metrics
import numpy as np
from scipy.stats import gaussian_kde
from statsmodels.distributions.empirical_distribution import ECDF
from torch.utils.data import TensorDataset
!pip install aif360
!pip install fairlearn
from fairlearn.metrics import demographic_parity_difference
from fairlearn.metrics import equalized_odds_difference
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.1/214.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 3.7 MB/s eta 0:00:00


In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, include_demographic_s=False, include_demographic_t=False, bootstrap=False):
        if bootstrap:
            data = self.resample(data)
        self.data = data

        # Decide whether to include the demographic variable for gender and race
        self.sensitive_t = data['gender:is_Male'].values if include_demographic_t else None
        self.sensitive_s = data['race:is_White'].values if include_demographic_s else None

        # Drop unnecessary columns
        self.features = data.drop(['income:is_>50K', 'gender:is_Male', 'race:is_White'], axis=1).values
        self.labels = data['income:is_>50K'].values

    def __len__(self):
            return len(self.data)

    def resample(self, data):
        # Perform bootstrapping by sampling with replacement
        return data.sample(n=len(data), replace=True, random_state=42)

    def __getitem__(self, idx):
        result = {
            'features': torch.tensor(self.features[idx], dtype=torch.float),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }
        if self.sensitive_s is not None:
            result['sensitive_s'] = torch.tensor(self.sensitive_s[idx], dtype=torch.float)
        if self.sensitive_t is not None:
            result['sensitive_t'] = torch.tensor(self.sensitive_t[idx], dtype=torch.float)
        return result

# Read the CSV file
data = pd.read_csv('adult_norm copy.csv')
data['race:is_White'] = data['race:is_White'].astype(int)

# Drop the other race-related columns
data.drop(columns=['race:is_Other', 'race:is_Black', 'race:is_Asian-Pac-Islander'], inplace=True)

# Split into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Create datasets with bootstrapping
train_dataset = MyDataset(train_data, include_demographic_s=True, include_demographic_t=True, bootstrap=True)
test_dataset = MyDataset(test_data, include_demographic_s=True, include_demographic_t=False, bootstrap=True)



In [ ]:
#dataloader = DataLoader(dataset, batch_size=32)  # 设置 batch_size

#from torch.utils.data import random_split

# Determine the lengths of your splits
# 80 % and 20%
# 保证 X and Y



#lengths = [int(len(dataset)*0.8), len(dataset) - int(len(dataset)*0.8)]

# Use random_split() to split the dataset
#train_dataset, test_dataset = random_split(dataset, lengths)


In [ ]:
# Load data using dataloader

data_loader_train = torch.utils.data.DataLoader(dataset=train_dataset,
                                                batch_size = 64,
                                                shuffle = True)

data_loader_test = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=64,
                                               shuffle=False)

In [ ]:
# Print out the baches
for i, data in enumerate(data_loader_train):
    print(data)
    if i > 5:  # only print out the first 5 batches
        break

# Print out the data in the test set
for i, data in enumerate(data_loader_test):
    print(data)
    if i > 5:  # only print out the first 5 batches
        break

{'features': tensor([[0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 0.2020],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.3030],
        [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.4040],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0091, 0.0000, 0.4040],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.4040],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.4040]]), 'label': tensor([0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0]), 'sensitive_s': tensor([1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1.]), 'sensitive_t': tensor([1., 0., 1., 0., 0., 1., 1., 1., 1

In [ ]:
# Formula to calculate phi(t)

# Calculate the proportion of `gender:is_Male` in the training set
"""prob_train = train_dataset.demographic.mean()

# Calculate the proportion of `gender:is_Male` in the test set
prob_test = test_dataset.demographic.mean()

# Calculate the ratio
phi = prob_test / prob_train

print(f"The ratio φ(t) = {phi}")"""



'prob_train = train_dataset.demographic.mean()\n\n# Calculate the proportion of `gender:is_Male` in the test set\nprob_test = test_dataset.demographic.mean()\n\n# Calculate the ratio\nphi = prob_test / prob_train\n\nprint(f"The ratio φ(t) = {phi}")'

In [ ]:
"""class FairnessAwareLoss(nn.Module):
    def __init__(self, model, lambda_fairness):
        super(FairnessAwareLoss, self).__init__()
        self.model = model
        self.lambda_fairness = lambda_fairness

    def forward(self, features, labels, S):
        outputs = self.model(features)
        cross_entropy_loss = nn.CrossEntropyLoss()(outputs, labels)

        fairness_metric = self.calculate_fairness_metric(outputs, S)
        fairness_penalty = self.lambda_fairness * fairness_metric

        total_loss = cross_entropy_loss + fairness_penalty
        return total_loss

    def calculate_fairness_metric(self, outputs, S):
        # Implement or call a function to calculate the fairness metric
        # based on the outputs and the sensitive attribute S
        outputs_bin = (torch.sigmoid(outputs) > 0.5).float()  # convert to binary predictions
        mask_s0 = (S == 0).float()
        mask_s1 = (S == 1).float()

        parity_s0 = torch.mean(mask_s0 * outputs_bin) / torch.clamp(torch.sum(mask_s0), min=1)
      parity_s1 = torch.mean(mask_s1 * outputs_bin) / torch.clamp(torch.sum(mask_s1), min=1)

    fairness_metric = torch.abs(parity_s0 - parity_s1)

        return fairness_metric"""

'class FairnessAwareLoss(nn.Module):\n    def __init__(self, model, lambda_fairness):\n        super(FairnessAwareLoss, self).__init__()\n        self.model = model\n        self.lambda_fairness = lambda_fairness\n\n    def forward(self, features, labels, S):\n        outputs = self.model(features)\n        cross_entropy_loss = nn.CrossEntropyLoss()(outputs, labels)\n\n        fairness_metric = self.calculate_fairness_metric(outputs, S)\n        fairness_penalty = self.lambda_fairness * fairness_metric\n\n        total_loss = cross_entropy_loss + fairness_penalty\n        return total_loss\n\n    def calculate_fairness_metric(self, outputs, S):\n        # Implement or call a function to calculate the fairness metric\n        # based on the outputs and the sensitive attribute S\n        outputs_bin = (torch.sigmoid(outputs) > 0.5).float()  # convert to binary predictions\n        mask_s0 = (S == 0).float()\n        mask_s1 = (S == 1).float()\n\n        parity_s0 = torch.mean(mask_s0 * o

In [ ]:
class FairnessAwareLoss(nn.Module):
    def __init__(self, model, epsilon_dp, lambda_penalty):
        super(FairnessAwareLoss, self).__init__()
        self.model = model
        self.epsilon_dp = epsilon_dp
        self.lambda_b = lambda_penalty

    def forward(self, features, labels, S):

        # Calculate the classification error
        outputs = self.model(features)
        cross_entropy = nn.CrossEntropyLoss()
        loss_classif = cross_entropy(outputs, labels)

        # Calculate the demographic parity penalty (Penalty())
        lambda_b = 1000000
        #mask_s0 = (S == 0).float()

        #mask_s1 = (S == 1).float()
        outputs_prob = torch.softmax(outputs, dim=1)
        positive_prob_s0 = outputs_prob[S == 0, 1].mean()
        positive_prob_s1 = outputs_prob[S == 1, 1].mean()
        gDP = torch.abs(positive_prob_s0 - positive_prob_s1) - self.epsilon_dp
        penalty = self.lambda_b * torch.relu(gDP)
        #print(f"S: {S}")
        #print(f"outputs_prob: {outputs_prob}")
        #print(f"positive_prob_s0: {positive_prob_s0}")
        #print(f"positive_prob_s1: {positive_prob_s1}")
        #print(f"gDP: {gDP}")
        #print(f"penalty: {penalty}")
        #E_s0 = torch.sum(mask_s0 * outputs) / torch.clamp(torch.sum(mask_s0), min=1)
        #E_s1 = torch.sum(mask_s1 * outputs) / torch.clamp(torch.sum(mask_s1), min=1)
        #gDP = torch.mean(E_s0 - E_s1 - self.epsilon_dp)
        # Combine the worst-case classification error and the penalty
        total_loss = loss_classif + penalty
        '''print("loss_classif:", loss_classif.item())
        print("positive_prob_s0:", positive_prob_s0.item())
        print("positive_prob_s1:", positive_prob_s1.item())
        print("gDP:", gDP.item())
        print("penalty:", penalty.item())
        print("total_loss:", total_loss.item())'''
        return total_loss, penalty



NameError: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. Define the MLP
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
        self.layer2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = torch.relu(self.layer1(x))  # sigmoid is applied once here
        out = self.layer2(out)
        return out



In [ ]:
# Function to perform one set of trials
def run_experiment(lambda_penalty, num_trials=5, num_epochs=50):
    all_accuracies = []
    all_demographic_differences = []

    for trial in range(num_trials): # Loop over trials
        loss_values = []
        model = MLP(36, 50, 2)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
        fairness_loss = FairnessAwareLoss(model, epsilon_dp=0.01, lambda_penalty=lambda_penalty)
        for epoch in range(num_epochs):
          epoch_loss = 0
          num_batches = 0
          for i, batch in enumerate(data_loader_train):
              features = batch['features']
              labels = batch['label']
              sensitive_s = batch['sensitive_s']  # Extracted from the batch
              optimizer.zero_grad()
              loss, penalty = fairness_loss(features, labels, sensitive_s)  # Using sensitive_s instead of sensitive
              loss.backward()
              optimizer.step()
              #print(f'Epoch {epoch}, Penalty {penalty.item()}')
              epoch_loss += loss.item()
              num_batches += 1
          loss_values.append(epoch_loss / num_batches)
          print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {epoch_loss/num_batches}')

        # Evaluate the model
        model.eval()
        correct = 0
        total = 0
        demographic_differences = []
        with torch.no_grad():
            for batch in data_loader_test:
                features = batch['features']
                labels = batch['label']
                sensitive_s = batch['sensitive_s']  # or another key if you use a different one in your test data
                outputs = model(features)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                y_pred = torch.argmax(outputs, dim=1).cpu().numpy()
                y_true = labels.cpu().numpy()
                sensitive_features = sensitive_s.cpu().numpy()
                difference = demographic_parity_difference(y_true, y_pred, sensitive_features=sensitive_features)
                demographic_differences.append(difference)
        average_demographic_difference = sum(demographic_differences) / len(demographic_differences) # Moved outside loop
        accuracy = 100 * correct / total
        print(f'Trial {trial+1}: Average Demographic Difference: {average_demographic_difference}')
        print(f'Trial {trial+1}: Accuracy: {accuracy}')
        all_accuracies.append(accuracy)
        all_demographic_differences.append(average_demographic_difference)


    return all_accuracies, all_demographic_differences
# Run experiments with and without penalty
lambda_penalty_with = 1
lambda_penalty_without = 0.0
accuracies_with, differences_with = run_experiment(lambda_penalty_with, num_trials=5)
accuracies_without, differences_without = run_experiment(lambda_penalty_without, num_trials=5)

# Plot demographic parity graph
num_trials = 5
plt.figure(figsize=(12, 6))
plt.plot(range(1, num_trials + 1), differences_with, 'b-', linewidth=2, markersize=6, label="With Penalty")
plt.plot(range(1, num_trials + 1), differences_without, 'r-', linewidth=2, markersize=6, label="Without Penalty")
plt.xlabel('Trial', fontsize=14)
plt.ylabel('Demographic Parity', fontsize=14)
plt.title('Demographic Parity vs Trial', fontsize=16)
plt.grid(True, linestyle='--')
plt.legend(fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

# Plot accuracy graph
plt.figure(figsize=(12, 6))
plt.plot(range(1, num_trials + 1), accuracies_with, 'b-', linewidth=2, markersize=6, label="With Penalty")
plt.plot(range(1, num_trials + 1), accuracies_without, 'r-', linewidth=2, markersize=6, label="Without Penalty")
plt.xlabel('Trial', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.title('Accuracy vs Trial', fontsize=16)
plt.grid(True, linestyle='--')
plt.legend(fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Set y-axis limits to minimize the appearance of the difference
plt.ylim(0, 100) # Set y-axis limits from 0 to 100
plt.show()
